In [21]:
from collections import namedtuple
from dataclasses import dataclass
import re

In [53]:
@dataclass
class Valve:
    name: str
    flowrate: int
    tunnels: list[str]
    
    pat = re.compile(r"Valve (.+) has flow rate=(\d+); tunnels? leads? to valves? (.+)")
    
    @classmethod
    def parse(cls, s):
        m = cls.pat.match(s)
        if m:
            valve = m.group(1)
            flowrate = int(m.group(2))
            tunnels = m.group(3).split(", ")
            return cls(valve, flowrate, tunnels)

@dataclass(frozen=True)
class State:
    valve: str
    openvalves: frozenset[str]

Step = namedtuple("Step", "valve minute openvalves")
        
class ValveSeq:
    def __init__(self, valvelist):
        self.vlist = valvelist
        self.vindex = {}
        for v in self.vlist:
            self.vindex[v.name] = v
            
    def get(self, name):
        return self.vindex[name]
    
    def totalpressure(self, openvalves):
        return sum(self.vindex[k].flowrate * (30 - v) for k, v in openvalves.items())
    
    def traverse(self, startvalve, debug=False):
        queue = []
        queue.append(Step(startvalve, 1, {}))
        visited = set()
        visited.add(State(startvalve, frozenset()))
        
        maxstep = None
        maxpressure = 0
        
        while queue:
            s = queue.pop(0)
            if s.minute > 30:
                return maxpressure, maxstep
            pressure = self.totalpressure(s.openvalves)
            if debug:
                if "DD" in s.openvalves and s.openvalves["DD"] == 2 and "BB" in s.openvalves and s.openvalves["BB"] == 5:
                    print(f"{pressure:5d} {s}")
            if pressure > maxpressure:
                maxpressure = pressure
                maxstep = s
            neighbors = []
            if s.valve not in s.openvalves and self.get(s.valve).flowrate > 0:
                # don't bother opening valves that don't release pressure
                neighbors.append(Step(s.valve, s.minute+1, s.openvalves | {s.valve: s.minute}))
            for nvalve in self.get(s.valve).tunnels:
                neighbors.append(Step(nvalve, s.minute+1, s.openvalves))
            for ns in neighbors:
                nstate = State(ns.valve, frozenset(ns.openvalves.keys()))
                if nstate not in visited:
                    visited.add(nstate)
                    queue.append(ns)
    
        return maxpressure, maxstep
    
    @classmethod
    def readfile(cls, filename):
        out = []
        with open(filename, "r") as fh:
            for s in fh:
                out.append(Valve.parse(s.strip()))
        return cls(out)

test = ValveSeq.readfile("test.txt")
for v in test.vlist:
    print(v)
print()
test.traverse("AA", debug=True)

Valve(name='AA', flowrate=0, tunnels=['DD', 'II', 'BB'])
Valve(name='BB', flowrate=13, tunnels=['CC', 'AA'])
Valve(name='CC', flowrate=2, tunnels=['DD', 'BB'])
Valve(name='DD', flowrate=20, tunnels=['CC', 'AA', 'EE'])
Valve(name='EE', flowrate=3, tunnels=['FF', 'DD'])
Valve(name='FF', flowrate=0, tunnels=['EE', 'GG'])
Valve(name='GG', flowrate=0, tunnels=['FF', 'HH'])
Valve(name='HH', flowrate=22, tunnels=['GG'])
Valve(name='II', flowrate=0, tunnels=['AA', 'JJ'])
Valve(name='JJ', flowrate=21, tunnels=['II'])

  885 Step(valve='BB', minute=6, openvalves={'DD': 2, 'BB': 5})
  885 Step(valve='CC', minute=7, openvalves={'DD': 2, 'BB': 5})
  885 Step(valve='AA', minute=7, openvalves={'DD': 2, 'BB': 5})
  885 Step(valve='II', minute=8, openvalves={'DD': 2, 'BB': 5})
  885 Step(valve='JJ', minute=9, openvalves={'DD': 2, 'BB': 5})
 1326 Step(valve='JJ', minute=10, openvalves={'DD': 2, 'BB': 5, 'JJ': 9})
 1326 Step(valve='II', minute=11, openvalves={'DD': 2, 'BB': 5, 'JJ': 9})
 1326 Step(valve=

(1628,
 Step(valve='HH', minute=17, openvalves={'JJ': 3, 'BB': 7, 'DD': 10, 'EE': 12, 'HH': 16}))

In [17]:
@dataclass(frozen=True)
class State:
    valve: str
    openvalves: frozenset[str]
    
    def neighbors(self, valveseq):


s = State("AA", frozenset())
print(s)
for ns in s.neighbors(test):
    print(ns)

State(valve='AA', openvalves=frozenset())
State(valve='DD', openvalves=frozenset())
State(valve='II', openvalves=frozenset())
State(valve='BB', openvalves=frozenset())
